In [28]:
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_algorithms
!pip install qiskit_machine_learning

In [29]:
import qiskit
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit import transpile

In [30]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

In [31]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

def read_data(directory, target_size = (300, 300)):
    all_files = os.listdir(directory)
    not_mask_images = sorted([file for file in all_files if "mask" not in file])
    mask_images = sorted([file for file in all_files if "mask" in file])
    images = []
    masks = []
    temp_masks = []

    for file_name in not_mask_images:
        image = cv2.imread(os.path.join(directory, file_name))
        images.append(cv2.resize(image, target_size))

    prev_file = None
    for file_name in mask_images:
        image = cv2.imread(os.path.join(directory, file_name), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, target_size)
        if prev_file != None and (str(prev_file).replace(".png", "").replace(" ", "_") in str(file_name).replace(".png", "").replace(" ","_")):
            masks[-1] += image
        else:
            temp_masks.append(file_name)
            masks.append(image)
            prev_file = file_name

    return np.array(images), np.array(masks)

def show_image(data, index):
    image = data[index]
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW, Adam
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, Dataset
import pickle
import numpy as np
import pandas as pd

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
class UNet(nn.Module):
    def __init__(self, base_model, qnn):
        super(UNet, self).__init__()
        self.model = base_model
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc1 = nn.Linear(28900, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(128, 8)
        self.relu4 = nn.ReLU()
        self.qnn = TorchConnector(qnn)
        self.output = nn.Linear(1, 3)

    def forward(self, x):
        segmented = self.model(x)
        concatenated = torch.cat((x, segmented), dim=1)
        classified = self.maxpool(concatenated)
        classified = classified.view(classified.size(0), -1)
        classified = self.fc1(classified)
        classified = self.relu1(classified)
        classified = self.fc2(classified)
        classified = self.relu2(classified)
        classified = self.fc3(classified)
        classified = self.relu3(classified)
        classified = self.fc4(classified)
        classified = self.relu4(classified)
        classified = self.qnn(classified)
        output = self.output(classified)
        return segmented, output

def create_qnn():
    feature_map = ZZFeatureMap(8)
    ansatz = RealAmplitudes(8, reps=1)
    qc = QuantumCircuit(8)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn


U_qnn = create_qnn()

In [37]:
unet = UNet(model, U_qnn).to(device)
print(unet)

UNet(
  (model): UNet(
    (encoder1): Sequential(
      (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc1relu1): ReLU(inplace=True)
      (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc1relu2): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (encoder2): Sequential(
      (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc2relu1): ReLU(inplace=True)
      (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (enc2norm2): BatchNorm2

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Example usage:
total_params = count_parameters(unet)
print("Total trainable parameters: ", total_params)

Total trainable parameters:  22725631


In [39]:
NUM_EPOCHS = 50
BATCH_SIZE = 16

In [40]:
benign_images, benign_masks = read_data(directory = "/content/drive/MyDrive/data/benign", target_size=(256, 256))
normal_images, normal_masks = read_data(directory = "/content/drive/MyDrive/data/normal", target_size=(256, 256))
malignant_images, malignant_masks = read_data(directory = "/content/drive/MyDrive/data/malignant", target_size=(256, 256))

benign_masks = np.reshape(benign_masks, (benign_masks.shape[0], 1, benign_masks.shape[1], benign_masks.shape[2]))
normal_masks = np.reshape(normal_masks, (normal_masks.shape[0], 1, normal_masks.shape[1], normal_masks.shape[2]))
malignant_masks = np.reshape(malignant_masks, (malignant_masks.shape[0], 1, malignant_masks.shape[1], malignant_masks.shape[2]))

benign_images = np.transpose(benign_images, (0, 3, 1, 2))
normal_images = np.transpose(normal_images, (0, 3, 1, 2))
malignant_images = np.transpose(malignant_images, (0, 3, 1, 2))

benign_values = np.array([1, 0, 0], dtype = np.float32)
normal_values = np.array([0, 1, 0], dtype = np.float32)
malignant_values = np.array([0, 0, 1], dtype = np.float32)
num_benign = len(benign_images)
num_normal = len(normal_images)
num_malignant = len(malignant_images)
y_classification = []
y_segmentation = []
X = []

for i in range(num_benign):
    y_classification.append(benign_values)
for i in range(num_normal):
    y_classification.append(normal_values)
for i in range(num_malignant):
    y_classification.append(malignant_values)
y_classification = np.array(y_classification, dtype = np.float32)

benign_masks[benign_masks>0] = 1
normal_masks[normal_masks>0] = 1
malignant_masks[malignant_masks>0] = 1
benign_images = benign_images.astype(dtype = np.float32)
normal_images = normal_images.astype(dtype = np.float32)
malignant_images = malignant_images.astype(dtype = np.float32)
benign_images /= 255
normal_images /= 255
malignant_images /= 255

for img in benign_images:
    X.append(img)
for img in normal_images:
    X.append(img)
for img in malignant_images:
    X.append(img)
X = np.array(X, dtype=np.float32)

for img in benign_masks:
    y_segmentation.append(img)
for img in normal_masks:
    y_segmentation.append(img)
for img in malignant_masks:
    y_segmentation.append(img)
y_segmentation = np.array(y_segmentation, dtype=np.float32)

num_samples = len(y_segmentation)
perm = np.random.permutation(num_samples)

X = X[perm]
y_classification = y_classification[perm]
y_segmentation = y_segmentation[perm]

total_samples = len(X)
train_size = int(0.7 * total_samples)
val_size = int(0.2 * total_samples)

X_train = X[:train_size]
y_classification_train = y_classification[:train_size]
y_segmentation_train = y_segmentation[:train_size]

X_val = X[train_size:train_size + val_size]
y_classification_val = y_classification[train_size:train_size + val_size]
y_segmentation_val = y_segmentation[train_size:train_size + val_size]

X_test = X[train_size + val_size:]
y_classification_test = y_classification[train_size + val_size:]
y_segmentation_test = y_segmentation[train_size + val_size:]

In [41]:
print(len(benign_images) + len(benign_masks) + len(normal_images) + len(normal_masks) + len(malignant_images) + len(malignant_masks))

1560


In [42]:
len(benign_images) + len(benign_masks)

874

In [43]:
len(normal_images) + len(normal_masks)

266

In [44]:
len(malignant_images) + len(malignant_masks)

420

In [45]:
class CustomDataset(Dataset):
    def __init__(self, X, y1, y2):
        self.X = X
        self.y1 = y1
        self.y2 = y2

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y1[idx], self.y2[idx]

In [46]:
X_train = torch.Tensor(X_train).to(device)
y_classification_train = torch.Tensor(y_classification_train).to(device)
y_segmentation_train = torch.Tensor(y_segmentation_train).to(device)

X_val = torch.Tensor(X_val).to(device)
y_classification_val = torch.Tensor(y_classification_val).to(device)
y_segmentation_val = torch.Tensor(y_segmentation_val).to(device)

X_test = torch.Tensor(X_test).to(device)
y_classification_test = torch.Tensor(y_classification_test).to(device)
y_segmentation_test = torch.Tensor(y_segmentation_test).to(device)

train_dataset = CustomDataset(X_train, y_classification_train, y_segmentation_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_dataset = CustomDataset(X_val, y_classification_val, y_segmentation_val)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataset = CustomDataset(X_test, y_classification_test, y_segmentation_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [47]:
y_segmentation_train.shape
y_classification.shape

(780, 3)

In [48]:
unet.train()
lr = 1e-2
b1 = 0.7
b2 = 0.999
optimizer = Adam(unet.parameters(), lr=lr, betas=(b1, b2), weight_decay=0.005)

segmentation_loss = torch.nn.MSELoss()
classification_loss = nn.CrossEntropyLoss()

classification_loss_train = []
segmentation_loss_train = []
classification_loss_val = []
segmentation_loss_val = []
classification_loss_test = []
segmentation_loss_test = []
accuracy_list = []
val_accuracy_list = []
test_accuracy_list = []

for i in range(NUM_EPOCHS):
    unet.train()
    train_loss_class = []
    train_loss_seg = []
    val_loss_class = []
    val_loss_seg = []
    test_loss_class = []
    test_loss_seg = []
    accuracy = []
    val_accuracy = []
    test_accuracy = []

    for batch in train_loader:
        X_data, y_class, y_seg = batch
        optimizer.zero_grad()
        segmentation_output, classification_output = unet(X_data)
        segmentation_output = torch.sigmoid(segmentation_output)
        classification_output = torch.softmax(classification_output, dim=1)
        seg_loss = segmentation_loss(segmentation_output, y_seg)
        class_loss = classification_loss(classification_output, y_class)
        loss = seg_loss + class_loss
        loss.backward()
        optimizer.step()
        train_loss_class.append(class_loss.item())
        train_loss_seg.append(seg_loss.item())
        predicted_classes = torch.argmax(classification_output, dim=1)
        temp_class = torch.argmax(y_class, dim=1)
        accuracy.append((predicted_classes == temp_class).float().mean().item())

    accuracy = sum(accuracy)/len(accuracy)
    accuracy_list.append(accuracy)
    classification_loss_train.append(sum(train_loss_class) / len(train_loss_class))
    segmentation_loss_train.append(sum(train_loss_seg) / len(train_loss_seg))

    unet.eval()
    with torch.no_grad():
        val_loss_class = []
        val_loss_seg = []
        for batch in val_loader:
            X_val, y_class_val, y_seg_val = batch
            segmentation_output_val, classification_output_val = unet(X_val)
            segmentation_output_val = torch.sigmoid(segmentation_output_val)
            classification_output_val = torch.softmax(classification_output_val, dim=1)
            seg_loss_val = segmentation_loss(segmentation_output_val, y_seg_val)
            class_loss_val = classification_loss(classification_output_val, y_class_val)
            val_loss_class.append(class_loss_val.item())
            val_loss_seg.append(seg_loss_val.item())
            predicted_classes = torch.argmax(classification_output_val, dim=1)
            temp_class = torch.argmax(y_class_val, dim=1)
            val_accuracy.append((predicted_classes == temp_class).float().mean().item())

        val_accuracy = sum(val_accuracy)/len(val_accuracy)
        val_accuracy_list.append(val_accuracy)
        classification_loss_val.append(sum(val_loss_class) / len(val_loss_class))
        segmentation_loss_val.append(sum(val_loss_seg) / len(val_loss_seg))

    with torch.no_grad():
        test_loss_class = []
        test_loss_seg = []
        for batch in test_loader:
            X_test, y_class_test, y_seg_test = batch
            segmentation_output_test, classification_output_test = unet(X_test)
            segmentation_output_test = torch.sigmoid(segmentation_output_test)
            classification_output_test = torch.softmax(classification_output_test, dim=1)
            seg_loss_test = segmentation_loss(segmentation_output_test, y_seg_test)
            class_loss_test = classification_loss(classification_output_test, y_class_test)
            test_loss_class.append(class_loss_test.item())
            test_loss_seg.append(seg_loss_test.item())
            predicted_classes = torch.argmax(classification_output_test, dim=1)
            temp_class = torch.argmax(y_class_test, dim=1)
            test_accuracy.append((predicted_classes == temp_class).float().mean().item())

        test_accuracy = sum(test_accuracy)/len(test_accuracy)
        test_accuracy_list.append(test_accuracy)
        classification_loss_test.append(sum(test_loss_class) / len(test_loss_class))
        segmentation_loss_test.append(sum(test_loss_seg) / len(test_loss_seg))

    print(f"Epoch: {i}, Classification Training Loss: {classification_loss_train[-1]}, Train accuracy: {accuracy_list[-1]}, Val Accuracy: {val_accuracy_list[-1]}, Test Accuracy: {test_accuracy_list[-1]} Segmentation Training Loss: {segmentation_loss_train[-1]}, Classification Validation Loss: {classification_loss_val[-1]}, Segmentation Validation Loss: {segmentation_loss_val[-1]}, Classification Test Loss: {classification_loss_test[-1]}, Segmentation Test Loss: {segmentation_loss_test[-1]}")


KeyboardInterrupt: 

In [ ]:
torch.save(unet.state_dict(), 'q_unet.pth')

In [ ]:
lists_to_save = {
    'classification_loss_train': classification_loss_train,
    'segmentation_loss_train': segmentation_loss_train,
    'classification_loss_val': classification_loss_val,
    'segmentation_loss_val': segmentation_loss_val,
    'classification_loss_test': classification_loss_test,
    'segmentation_loss_test': segmentation_loss_test,
    'accuracy_list': accuracy_list,
    'val_accuracy_list': val_accuracy_list,
    'test_accuracy_list': test_accuracy_list
}

with open('q_unet_results.pkl', 'wb') as f:
    pickle.dump(lists_to_save, f)

---

In [49]:
import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F

In [50]:
def create_qnn():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn


qnn4 = create_qnn()

In [64]:
class Net(Module):
    def __init__(self, qnn):
        super().__init__()
        self.conv1 = Conv2d(3, 8, kernel_size=5)
        self.conv2 = Conv2d(8, 16, kernel_size=5)
        self.dropout = Dropout2d()
        self.fc1 = Linear(59536, 64)
        self.fc2 = Linear(64, 2)  # 2-dimensional input to QNN
        self.qnn = TorchConnector(qnn)  # Apply torch connector, weights chosen
        # uniformly at random from interval [-1,1].
        self.fc3 = Linear(1, 3)  # 1-dimensional output from QNN

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.qnn(x)  # apply QNN
        x = self.fc3(x)
        x = torch.softmax(x, dim=1)
        return x


model4 = Net(qnn4).to(device)

In [65]:
optimizer = optim.Adam(model4.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

# Start training
epochs = 100  # Set number of epochs
loss_list = []  # Store loss history
model4.train()  # Set model to training mode

for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target, _) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model4(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]))

Training [1%]	Loss: 1.1435
Training [2%]	Loss: 1.1425
Training [3%]	Loss: 1.1371
Training [4%]	Loss: 1.1233
Training [5%]	Loss: 1.1286
Training [6%]	Loss: 1.1177
Training [7%]	Loss: 1.1193
Training [8%]	Loss: 1.1031
Training [9%]	Loss: 1.1034
Training [10%]	Loss: 1.1023
Training [11%]	Loss: 1.0910
Training [12%]	Loss: 1.0938
Training [13%]	Loss: 1.0780
Training [14%]	Loss: 1.0792
Training [15%]	Loss: 1.0757
Training [16%]	Loss: 1.0609
Training [17%]	Loss: 1.0433
Training [18%]	Loss: 1.0330
Training [19%]	Loss: 1.0229
Training [20%]	Loss: 1.0133
Training [21%]	Loss: 0.9999
Training [22%]	Loss: 0.9924
Training [23%]	Loss: 0.9824
Training [24%]	Loss: 0.9571
Training [25%]	Loss: 0.9474
Training [26%]	Loss: 0.9303
Training [27%]	Loss: 0.9144
Training [28%]	Loss: 0.8938
Training [29%]	Loss: 0.8923
Training [30%]	Loss: 0.8789
Training [31%]	Loss: 0.8574
Training [32%]	Loss: 0.8624
Training [33%]	Loss: 0.8631
Training [34%]	Loss: 0.8376
Training [35%]	Loss: 0.8233
Training [36%]	Loss: 0.9935
T

In [66]:
val_accuracy = []
with torch.no_grad():
    val_loss_class = []
    val_loss_seg = []
    for batch in val_loader:
        X_val, y_class_val, y_seg_val = batch
        output = model4(X_val)
        predicted_classes = torch.argmax(output, dim=1)
        temp_class = torch.argmax(y_class_val, dim=1)
        val_accuracy.append((predicted_classes == temp_class).float().mean().item())

    val_accuracy = sum(val_accuracy)/len(val_accuracy)
    print(val_accuracy)

0.6395833373069764


In [67]:
val_accuracy = []
with torch.no_grad():
    train_loss_class = []
    train_loss_seg = []
    for batch in train_loader:
        X_train, y_class_train, y_seg_train = batch
        output = model4(X_train)
        predicted_classes = torch.argmax(output, dim=1)
        temp_class = torch.argmax(y_class_train, dim=1)
        val_accuracy.append((predicted_classes == temp_class).float().mean().item())

    val_accuracy = sum(val_accuracy)/len(val_accuracy)
    print(val_accuracy)

0.825


In [68]:
torch.save(model4.state_dict(), 'model.pth')

with open('loss_list.pkl', 'wb') as f:
    pickle.dump(loss_list, f)

---

In [74]:
class Net(Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(3, 8, kernel_size=5)
        self.conv2 = Conv2d(8, 16, kernel_size=5)
        self.dropout = Dropout2d()
        self.fc1 = Linear(59536, 64)
        self.fc2 = Linear(64, 2)  # 2-dimensional input to QNN
        # uniformly at random from interval [-1,1].
        self.fc3 = Linear(2, 3)  # 1-dimensional output from QNN

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        x = torch.softmax(x, dim=1)
        return x


model4 = Net().to(device)

In [75]:
optimizer = optim.Adam(model4.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

# Start training
epochs = 100  # Set number of epochs
loss_list = []  # Store loss history
model4.train()  # Set model to training mode

for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target, _) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model4(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]))

Training [1%]	Loss: 0.9985
Training [2%]	Loss: 0.9908
Training [3%]	Loss: 0.9855
Training [4%]	Loss: 0.9918
Training [5%]	Loss: 0.9916
Training [6%]	Loss: 1.0037
Training [7%]	Loss: 0.9893
Training [8%]	Loss: 0.9914
Training [9%]	Loss: 0.9911
Training [10%]	Loss: 0.9911
Training [11%]	Loss: 0.9918
Training [12%]	Loss: 0.9902
Training [13%]	Loss: 0.9907
Training [14%]	Loss: 0.9905
Training [15%]	Loss: 0.9918
Training [16%]	Loss: 0.9915
Training [17%]	Loss: 0.9907
Training [18%]	Loss: 0.9900
Training [19%]	Loss: 0.9918
Training [20%]	Loss: 0.9907
Training [21%]	Loss: 0.9918
Training [22%]	Loss: 0.9926
Training [23%]	Loss: 0.9915
Training [24%]	Loss: 0.9907
Training [25%]	Loss: 0.9916
Training [26%]	Loss: 0.9918
Training [27%]	Loss: 0.9907
Training [28%]	Loss: 0.9907
Training [29%]	Loss: 0.9907
Training [30%]	Loss: 0.9907
Training [31%]	Loss: 0.9907
Training [32%]	Loss: 0.9900
Training [33%]	Loss: 0.9907
Training [34%]	Loss: 0.9907
Training [35%]	Loss: 0.9907
Training [36%]	Loss: 0.9907
T

In [78]:
val_accuracy = []
with torch.no_grad():
    val_loss_class = []
    val_loss_seg = []
    for batch in val_loader:
        X_val, y_class_val, y_seg_val = batch
        output = model4(X_val)
        predicted_classes = torch.argmax(output, dim=1)
        temp_class = torch.argmax(y_class_val, dim=1)
        val_accuracy.append((predicted_classes == temp_class).float().mean().item())

    val_accuracy = sum(val_accuracy)/len(val_accuracy)
    print(val_accuracy)

0.5354166686534881


In [76]:
val_accuracy = []
with torch.no_grad():
    train_loss_class = []
    train_loss_seg = []
    for batch in train_loader:
        X_train, y_class_train, y_seg_train = batch
        output = model4(X_train)
        predicted_classes = torch.argmax(output, dim=1)
        temp_class = torch.argmax(y_class_train, dim=1)
        val_accuracy.append((predicted_classes == temp_class).float().mean().item())

    val_accuracy = sum(val_accuracy)/len(val_accuracy)
    print(val_accuracy)

0.5607142857142857


In [79]:
torch.save(model4.state_dict(), 'not_quantum_model.pth')

with open('not_quantum_loss_list.pkl', 'wb') as f:
    pickle.dump(loss_list, f)